### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [112]:
import warnings
warnings.filterwarnings('ignore')

In [113]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data.loc[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((~data['ThoiGianKetThucLay'].isnull()) ^\
                          (~data['ThoiGianKetThucGiao'].isnull())))]
delivery['LoaiBacklog'] = 'Xử lý giao'
internal_delivery = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery, 'N0'] = delivery.loc[internal_delivery]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery, 'N0'] = delivery.loc[~internal_delivery]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(days=(120/24))
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(days=9999))


# return backlog (backlog trả)
returned = data.loc[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (~data['ThoiGianKetThucGiao'].isnull()))]
returned['LoaiBacklog'] = 'Xử lý trả'
internal_return = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return, 'N0'] = returned.loc[internal_return]['ThoiGianKetThucGiao']
returned.loc[~internal_return, 'N0'] = returned.loc[~internal_return]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(days=(72/24))
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(days=9999))


# pick_backlog (backlog lấy)
pickup = data[(data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))]
pickup['LoaiBacklog'] = 'Xử lý lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(days=3)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(days=9999))

# transit_backlog (backlog luân chuyển)
transit = data.loc[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]
transit['LoaiBacklog'] = 'Xử lý luân chuyển'
transit_filter = (transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh'] == 'Hồ Chí Minh')
transit['N0'] = transit['ThoiGianKetThucLay']
transit.loc[transit_filter, 'N+'] = transit.loc[transit_filter]['N0']+ pd.Timedelta(days=12/24)
transit.loc[~transit_filter, 'N+'] = transit.loc[~transit_filter]['N0'] + pd.Timedelta(days=32/24)
transit['Aging'] = (datetime.today() - transit['N+']).fillna(pd.Timedelta(days=9999))


# returned_transit_backlog (backlog luân chuyển trả hàng)
r_transit = data.loc[data['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))]
r_transit['LoaiBacklog'] = 'Xử lý luân chuyển trả'
r_transit_filter = (r_transit['DenTinh'] == 'Hà Nội') | (r_transit['DenTinh'] == 'Hồ Chí Minh')
r_transit['N0'] = r_transit['ThoiGianKetThucGiao']
r_transit.loc[r_transit_filter, 'N+'] = r_transit.loc[r_transit_filter]['N0'] + pd.Timedelta(days=36/24)
r_transit.loc[~r_transit_filter, 'N+'] = r_transit.loc[~r_transit_filter]['N0'] + pd.Timedelta(days=56/24)
r_transit['Aging'] = (datetime.today() - r_transit['N+']).fillna(pd.Timedelta(days=9999))


# composite data
data = delivery.append([returned, pickup, transit, r_transit])
data['e-commerces'] = np.nan

# name of e-commerces exchange
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# shopee_name['e-commerces'] = 'Shopee'
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# sendo_name['e-commerces'] = 'Sendo'
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# tiki_name['e-commerces'] = 'Tiki'
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'

# lazada_name['e-commerces'] = 'Lazada'
# data['e-commerces'] = shopee_name.append([sendo_name, tiki_name, lazada_name])

# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
# reset the index of dataframe
data.reset_index(inplace=True)
del data['index']
data[data['TrangThai'] == 'Tạo thành công']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,LoaiBacklog,N0,N+,Aging,e-commerces
3312,7FG3D7XA77,2396377322131456820,2020-07-18 10:29:42,2020-07-18 10:29:42,NaT,NaT,NaT,2020-07-21 23:00:00,NaT,NaN,Quận 7,Hồ Chí Minh,E,Thành phố Sơn La,Sơn La,A,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 280 Trần Đăng Ninh-Sơn La-Sơn La 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,6000,23x25x15,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,3 Ngày,1,0,0,NaN,1875980,NaN,'nguyenthikimsang8687',1154025,NaN,NaN,NaN,NaT,Xử lý lấy,2020-07-18 10:29:42,2020-07-21 10:29:42,29 days 09:17:54.552868,Shopee
3338,8FH36XN3U,NaN,2020-08-09 01:54:20,2020-08-09 08:00:00,NaT,NaT,NaT,2020-08-11 23:00:00,NaT,NaN,Quận 4,Hồ Chí Minh,E,Thị xã Thuận An,Bình Dương,F,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 7 Đường D20-Thuận An-Bình Dương,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,500,10x10x10,1292667,'11/08/2020 06:28:32 - GHN-PFA2A1|KHÔNG LIÊN L...,'',2 Ngày,6,0,0,NaN,NaN,NaN,'khách lẻ',1192664,NaN,NaN,NaN,NaT,Xử lý lấy,2020-08-09 01:54:20,2020-08-12 01:54:20,7 days 17:53:16.552868,NaN
3339,8FHDSRHQN,2412549316886528030,2020-08-09 18:00:34,2020-08-10 08:00:00,NaT,NaT,NaT,2020-08-12 23:00:00,NaT,NaN,Quận 7,Hồ Chí Minh,E,Quận Thanh Xuân,Hà Nội,G,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 57 Hạ Đình-Q.Thanh Xuân-HN 03,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,200,10x10x10,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,1,0,0,NaN,1875980,NaN,'hộp plastic',1154025,NaN,NaN,NaN,NaT,Xử lý lấy,2020-08-10 08:00:00,2020-08-13 08:00:00,6 days 11:47:36.552868,Shopee
3343,8FH1A6YA9,2411257597042688180,2020-08-07 23:14:09,2020-08-20 08:00:00,NaT,NaT,NaT,2020-08-22 23:00:00,NaT,NaN,Quận 7,Hồ Chí Minh,E,Thành phố Buôn Ma Thuột,Đắk Lắk,D,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 116 Hoàng Hoa Thám-Buôn Ma Thuột-Đắk L...,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,200,10x10x10,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,0,0,0,NaN,NaN,NaN,'sarangbomb.',1154025,NaN,NaN,NaN,NaT,Xử lý lấy,2020-08-20 08:00:00,2020-08-23 08:00:00,-4 days +11:47:36.552868,Shopee
3344,8FHSFYD5X,208QPPBT3C_1,2020-08-06 14:56:59,2020-08-06 14:56:59,NaT,NaT,NaT,2020-08-10 23:00:00,NaT,NaN,Quận 4,Hồ Chí Minh,E,Huyện Tân Biên,Tây Ninh,F,Bưu Cục 63 Vĩnh Hội-Q.4-HCM,Bưu Cục 2/9 Nguyễn Duy Trinh-Tân Biên-Tây Ninh,Bưu Cục 63 Vĩnh Hội-Q.4-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,1600,10x10x10,2534371,NaN,'',2 Ngày,1,0,0,NaN,2034362,NaN,'0919222230',1273286,NaN,NaN,NaN,NaT,Xử lý lấy,2020-08-06 14:56:59,2020-08-09 14:56:59,10 days 04:50:37.552868,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,8FH5DFQN1D,2418476422103040641,2020-08-17 22:16:41,2020-08-18 08:00:00,NaT,NaT,NaT,2020-08-20 23:00:00,NaT,NaN,Quận 7,Hồ Chí Minh,E,Thành phố Đà Lạt,Lâm Đồng,D,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 391 Nguyễn Tử Lực-Đà Lạt-Lâm Đồng,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Tạo thành công,450,10x10x10,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,0,0,0,NaN,NaN,NaN,'mammuoihouse',1154025,20QFB29734,2275.0,1626.0,2020-08-18 22:47:56,Xử lý lấy,2020-08-18 08:00:00,2020-08-21 08:00:00,-2 days +11:47:36.552868,Shopee
5632,8FHSA6LDA5,2418034510213120883,2020-08-17 07:38:41,2020-08-17 08:00:00,NaT,NaT,NaT,2020-08-18 23:00:00,NaT,NaN,Quận 7,Hồ Chí Minh,E,Quận Phú Nhuận,Hồ Chí Minh,

In [116]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'MaDH',
    'MaKH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'TGNhanKien',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'e-commerces',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(days=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val
                

# df_to_sheets(df, wks)


<Worksheet 'Backlog' id:181055436>

In [115]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)